In [1]:
import os
import json
from PIL import Image, ImageDraw
import re
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt
import tifffile as tiff
from tqdm import tqdm
from skimage.exposure import equalize_adapthist
from scipy.stats import stats
import matplotlib.animation as animation
import pandas as pd
import csv
import shutil
from skimage.morphology import dilation, erosion
from skimage import measure

from scipy.ndimage import center_of_mass
from glob import glob
import random

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import imageio
# from skimage import measure



def analyze_mask(frame, mask_path_template, size_threshold=100):


    def make_np(img):
        return np.array(img)


    def plot_labels(original, ax, title=None,
                    txt_args={'color':'red', 'ha':'center', 'va':'center', 
                              'fontsize':'x-large', 'fontweight':'bold'},
                    cmap='gist_earth'):
        ax.imshow(original, cmap=cmap)
        labels = np.unique(original)
        print("Labels in plot_labels function:", labels)
        ax.set_title(title)
        for label in labels:
            if label != 0:  # Skip background label
                inds = np.argwhere(original == label)
                loc = inds.mean(0)
                ax.text(loc[1], loc[0], str(label), **txt_args)

    
    # Load the mask image using imageio
    mask_path = mask_path_template.format(frame=frame)
    mask = imageio.imread(mask_path)

    # Print the labels directly from the mask
    unique_labels = np.unique(mask)
    print("Labels from the mask:", unique_labels)

    # Measure the properties of the labeled regions
    properties = measure.regionprops(mask)

    # Extract areas and labels, review small cell sizes
    labeled_areas = [(prop.label, prop.area) for prop in properties if prop.label != 0]
    labels, areas = zip(*labeled_areas)

    # Print details of very small cells
    print("Reviewing small cell sizes (less than 100 pixels):")
    for label, area in zip(labels, areas):
        if area < 100:
            print(f"Cell label: {label}, Cell size: {area} pixels")

    # Apply a size threshold to filter out small artifacts
    filtered_areas = [(prop.label, prop.area) for prop in properties if prop.label != 0 and prop.area >= size_threshold]
    filtered_labels, filtered_areas = zip(*filtered_areas)

    # Calculate the area of the background
    background_area = np.sum(mask == 0)

    # Compute the average, min, and max cell sizes for filtered cells
    average_cell_size_filtered = np.mean(filtered_areas)
    min_cell_size_filtered = np.min(filtered_areas)
    max_cell_size_filtered = np.max(filtered_areas)
    min_cell_label_filtered = filtered_labels[np.argmin(filtered_areas)]
    max_cell_label_filtered = filtered_labels[np.argmax(filtered_areas)]

    # Display filtered results and areas of individual cells
    print(f"\nNumber of cells after applying size threshold: {len(filtered_areas)}")
    print(f"Average cell size: {average_cell_size_filtered} pixels")
    print(f"Minimum cell size: {min_cell_size_filtered} pixels (Label: {min_cell_label_filtered})")
    print(f"Maximum cell size: {max_cell_size_filtered} pixels (Label: {max_cell_label_filtered})")
    print(f"Background size: {background_area} pixels(Label: 0)")
    # print(f"Background label: 0")

    # Print areas of individual cells
    print("\nIndividual cell areas after filtering:")
    for label, area in zip(filtered_labels, filtered_areas):
        print(f"Cell label: {label}, Cell area: {area} pixels")

    # Plot the labels for filtered cells, excluding the background label
    fig, ax = plt.subplots(figsize=(10, 10))
    plot_labels(mask, ax, title="Labeled Mask with Correct Labels")
    plt.show()

# Example usage
frame = 410
mask_path_template = "/home/MinaHossain/Label_5004/6row_images_min_A/01_ST/SEG/man_seg{frame:04d}.tif"
# mask_path_template = "/home/MinaHossain/Label_5004/6row_images_min_A/02_GT/SEG/man_seg{frame:04d}.tif"
analyze_mask(frame, mask_path_template)





In [ ]:

# === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE 

# import os
# import numpy as np
# import tifffile
# import cv2
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# === PARAMETERS ===
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS
output_video = "cell_4_tracking_colored_final.avi"
target_cell_id = 4
video_fps = 1
upscale_factor = 1.5

# === STEP 1: Build cell_locations + cell color map ===
cell_locations = {}
all_cell_ids = set()

# First pass: collect all cell IDs
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]
    all_cell_ids.update(cell_ids)

# Assign unique colors to each cell ID
cell_colors = {}
for cid in all_cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = (0, 0, 255)  # Bright RED for target cell
    else:
        cell_colors[cid] = tuple(random.randint(30, 220) for _ in range(3))  # Random distinct color

# Second pass: build centroid locations
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]

    cell_locations[image_name] = {}
    for cid in cell_ids:
        binary_mask = (mask == cid)
        if np.any(binary_mask):
            y, x = center_of_mass(binary_mask)
            cell_locations[image_name][cid] = (x, y)

# === STEP 2: Create video frames ===
frames = []
for frame_idx, image_name in enumerate(sorted(cell_locations.keys()), start=1):
    path = os.path.join(image_dir, image_name)
    mask = tifffile.imread(path)

    # Start with a black background
    color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)

    # Overlay each cell region with its assigned color
    for cid in np.unique(mask):
        if cid == 0:
            continue
        region = (mask == cid)
        color = cell_colors[cid]
        color_mask[region] = color

    # Add centroid markers and labels
    for cid, (x, y) in cell_locations[image_name].items():
        if cid == target_cell_id:
            # Large label for target cell
            cv2.circle(color_mask, (int(x), int(y)), 10, (255, 255, 255), -1)
            cv2.putText(color_mask, f"{cid}", (int(x)+10, int(y)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, cell_colors[cid], 3)
        else:
            # Regular label for other cells
            cv2.putText(color_mask, f"{cid}", (int(x)+10, int(y)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, cell_colors[cid], 2)

    # Add frame and file name info
    cv2.putText(color_mask, f"Frame {frame_idx:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
    cv2.putText(color_mask, image_name, (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

    # Upscale
    new_w = int(color_mask.shape[1] * upscale_factor)
    new_h = int(color_mask.shape[0] * upscale_factor)
    color_mask = cv2.resize(color_mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

    frames.append(color_mask)

# === STEP 3: Save final video ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Final video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final color-enhanced annotated video saved: {output_video}")
else:
    print("❌ No frames found.")


Final video frame size: 3072x6934
✅ Final color-enhanced annotated video saved: cell_4_tracking_colored_final.avi


In [5]:
# import os
# import numpy as np
# import tifffile
# import cv2
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# === PARAMETERS ===
# image_dir = "path/to/mask_images"  # <<< CHANGE THIS
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"
output_dir = "/home/MinaHossain/EmbedTrack/"  # <<< OUTPUT FOLDER
target_cell_id = 4
output_video = os.path.join(output_dir, f"cell_{target_cell_id}_zoomed_tracking.avi")
output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_frame_list.txt")
video_fps = 1
zoom_window_size = 256
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === STEP 1: Build cell_locations and color mapping ===
cell_locations = {}
target_frames = []
all_cell_ids = set()

mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))

# Collect all cell IDs
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]
    all_cell_ids.update(cell_ids)

# Assign unique colors
cell_colors = {}
for cid in all_cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = (0, 0, 255)  # Bright red
    else:
        cell_colors[cid] = tuple(random.randint(50, 220) for _ in range(3))

# Get centroid positions
for path in mask_paths:
    image_name = os.path.basename(path)
    mask = tifffile.imread(path)
    cell_ids = np.unique(mask)
    cell_ids = cell_ids[cell_ids != 0]

    cell_locations[image_name] = {}
    for cid in cell_ids:
        binary_mask = (mask == cid)
        if np.any(binary_mask):
            y, x = center_of_mass(binary_mask)
            cell_locations[image_name][cid] = (x, y)

# === STEP 2: Create Zoomed-in Frames ===
frames = []

for frame_idx, image_name in enumerate(sorted(cell_locations.keys()), start=1):
    if target_cell_id not in cell_locations[image_name]:
        continue

    target_frames.append((frame_idx, image_name))
    path = os.path.join(image_dir, image_name)
    mask = tifffile.imread(path)

    h, w = mask.shape
    cx, cy = cell_locations[image_name][target_cell_id]

    # Crop window
    half = zoom_window_size // 2
    x1 = int(max(cx - half, 0))
    y1 = int(max(cy - half, 0))
    x2 = int(min(cx + half, w))
    y2 = int(min(cy + half, h))

    cropped_mask = mask[y1:y2, x1:x2]
    color_crop = np.zeros((*cropped_mask.shape, 3), dtype=np.uint8)

    # Color each cell in crop
    for cid in np.unique(cropped_mask):
        if cid == 0:
            continue
        region = (cropped_mask == cid)
        color_crop[region] = cell_colors[cid]

    # Annotate labels
    for cid, (x, y) in cell_locations[image_name].items():
        if x < x1 or x > x2 or y < y1 or y > y2:
            continue
        rel_x, rel_y = int(x - x1), int(y - y1)
        if cid == target_cell_id:
            cv2.circle(color_crop, (rel_x, rel_y), 10, (255, 255, 255), -1)
            cv2.putText(color_crop, f"{cid}", (rel_x + 10, rel_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, cell_colors[cid], 3)
        else:
            cv2.putText(color_crop, f"{cid}", (rel_x + 10, rel_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, cell_colors[cid], 2)

    # Frame info
    cv2.putText(color_crop, f"Frame {frame_idx:03d}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    cv2.putText(color_crop, image_name, (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Target cell presence label (highlighted)
    cv2.putText(color_crop, f"Target Cell ID {target_cell_id} Visible",
                (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

    # Upscale
    color_crop = cv2.resize(color_crop, (int(color_crop.shape[1]*upscale_factor),
                                         int(color_crop.shape[0]*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(color_crop)

# === STEP 3: Write Video ===
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Zoomed-in annotated video saved: {output_video}")
else:
    print("❌ No frames found with target cell.")

# === STEP 4: Save Frame List ===
with open(output_frame_list_file, "w") as f:
    for frame_idx, image_name in target_frames:
        f.write(f"Frame {frame_idx:03d} - {image_name}\n")
print(f"✅ Frame list saved: {output_frame_list_file}")


✅ Zoomed-in annotated video saved: /home/MinaHossain/EmbedTrack/cell_4_zoomed_tracking.avi
✅ Frame list saved: /home/MinaHossain/EmbedTrack/cell_4_frame_list.txt


In [ ]:
# import os
# import numpy as np
# import tifffile
# import cv2
# import imageio
# import matplotlib.pyplot as plt
# from skimage import measure
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# # === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"  # <<< OUTPUT FOLDER
# target_cell_id = 4
# output_video = os.path.join(output_dir, f"cell_{target_cell_id}_only_video.avi")
# output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_accurate_frame_list.txt")
# video_fps = 1
# upscale_factor = 2

# # === Ensure output directory exists ===
# os.makedirs(output_dir, exist_ok=True)

# # === ANALYZE MASK FUNCTION (Simplified and Clean) ===
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         # plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

# # === MAIN PROCESSING ===
# mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
# cell_locations = {}
# target_frames = []
# all_cell_ids = set()

# # Collect all cell IDs and their centroid locations
# for path in mask_paths:
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     print(f"Checking {image_name} - Labels: {np.unique(mask)}")  # Diagnostic Print

#     cell_ids = np.unique(mask)
#     cell_ids = cell_ids[cell_ids != 0]
#     all_cell_ids.update(cell_ids)

#     cell_locations[image_name] = {}
#     for cid in cell_ids:
#         binary_mask = (mask == cid)
#         if np.any(binary_mask):
#             y, x = center_of_mass(binary_mask)
#             cell_locations[image_name][cid] = (x, y)

# # Assign distinct color to each cell
# cell_colors = {cid: (0, 0, 255) if cid == target_cell_id else tuple(random.randint(50, 230) for _ in range(3)) for cid in all_cell_ids}

# frames = []

# # Process only frames where target cell is present using cell_locations
# for frame_idx, path in enumerate(mask_paths, start=1):
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     # Use robust check
#     if target_cell_id not in cell_locations[image_name]:
#         print(f"Target cell ID {target_cell_id} NOT FOUND in {image_name}")  # Diagnostic
#         continue

#     print(f"✅ Target cell ID {target_cell_id} FOUND in {image_name}")  # Diagnostic
#     target_frames.append((frame_idx, image_name))
#     h, w = mask.shape
#     color_frame = np.zeros((h, w, 3), dtype=np.uint8)

#     for cid in np.unique(mask):
#         if cid == 0:
#             continue
#         region = (mask == cid)
#         color_frame[region] = cell_colors[cid]

#     for cid, (x, y) in cell_locations.get(image_name, {}).items():
#         label_size = 1.6 if cid == target_cell_id else 0.9
#         label_thickness = 3 if cid == target_cell_id else 2
#         label_color = cell_colors[cid]

#         if cid == target_cell_id:
#             cv2.circle(color_frame, (int(x), int(y)), 10, (255, 255, 255), -1)
#         cv2.putText(color_frame, f"{cid}", (int(x)+10, int(y)-10),
#                     cv2.FONT_HERSHEY_SIMPLEX, label_size, label_color, label_thickness)

#     cv2.putText(color_frame, f"Frame {frame_idx:03d}", (20, 50),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 4)
#     cv2.putText(color_frame, image_name, (20, 100),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255, 255, 255), 3)
#     cv2.putText(color_frame, f"Target Cell ID {target_cell_id} Visible", (20, 150),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 255), 3)

#     # Run analysis/plotting function
#     analyze_mask_from_path(mask_path=path, frame=frame_idx)

#     # Upscale
#     color_frame = cv2.resize(color_frame,
#                              (int(w * upscale_factor), int(h * upscale_factor)),
#                              interpolation=cv2.INTER_NEAREST)
#     frames.append(color_frame)

# # Save video
# if frames:
#     height, width, _ = frames[0].shape
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
#     for frame in frames:
#         out.write(frame)
#     out.release()
#     print(f"✅ Final video saved: {output_video}")
# else:
#     print("❌ No frames found for target cell.")

# # Save frame list
# with open(output_frame_list_file, "w") as f:
#     for frame_idx, image_name in target_frames:
#         f.write(f"Frame {frame_idx:03d} - {image_name}\n")

# print(f"✅ Accurate frame list saved: {output_frame_list_file}")
# print(f"✅ Total frames with target cell: {len(target_frames)}")


# Modify your tracking code to also store original mask label alongside assigned tracking ID in CSV?
# OR update your video generation code to use the CSV directly?


In [ ]:
# import os
# import numpy as np
# import tifffile
# import cv2
# import imageio
# import matplotlib.pyplot as plt
# from scipy.ndimage import center_of_mass
# from glob import glob
# import random

# # === PARAMETERS ===
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # <<< CHANGE THIS IF NEEDED
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
# target_cell_id = 4
# output_video = os.path.join(output_dir, f"cell_{target_cell_id}_only_video.avi")
# output_frame_list_file = os.path.join(output_dir, f"cell_{target_cell_id}_accurate_frame_list.txt")
# video_fps = 1
# upscale_factor = 2

# # === Ensure output directory exists ===
# os.makedirs(output_dir, exist_ok=True)

# # === ANALYZE MASK FUNCTION (Simplified and Clean) ===
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

# # === MAIN PROCESSING ===
# mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
# cell_locations = {}
# target_frames = []
# all_cell_ids = set()

# # Collect all cell IDs and their centroid locations
# for path in mask_paths:
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)
#     cell_ids = np.unique(mask)
#     cell_ids = cell_ids[cell_ids != 0]
#     all_cell_ids.update(cell_ids)

#     cell_locations[image_name] = {}
#     for cid in cell_ids:
#         binary_mask = (mask == cid)
#         if np.any(binary_mask):
#             y, x = center_of_mass(binary_mask)
#             cell_locations[image_name][cid] = (x, y)

# # Assign distinct color to each cell
# cell_colors = {cid: (0, 0, 255) if cid == target_cell_id else tuple(random.randint(50, 230) for _ in range(3)) for cid in all_cell_ids}

# frames = []

# # Process only frames where target cell is present
# for frame_idx, path in enumerate(mask_paths, start=1):
#     image_name = os.path.basename(path)
#     mask = tifffile.imread(path)

#     if target_cell_id not in np.unique(mask):
#         continue

#     target_frames.append((frame_idx, image_name))
#     h, w = mask.shape
#     color_frame = np.zeros((h, w, 3), dtype=np.uint8)

#     for cid in np.unique(mask):
#         if cid == 0:
#             continue
#         region = (mask == cid)
#         color_frame[region] = cell_colors[cid]

#     for cid, (x, y) in cell_locations.get(image_name, {}).items():
#         label_size = 1.6 if cid == target_cell_id else 0.9
#         label_thickness = 3 if cid == target_cell_id else 2
#         label_color = cell_colors[cid]

#         if cid == target_cell_id:
#             cv2.circle(color_frame, (int(x), int(y)), 10, (255, 255, 255), -1)
#         cv2.putText(color_frame, f"{cid}", (int(x)+10, int(y)-10),
#                     cv2.FONT_HERSHEY_SIMPLEX, label_size, label_color, label_thickness)

#     cv2.putText(color_frame, f"Frame {frame_idx:03d}", (20, 50),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 255, 255), 4)
#     cv2.putText(color_frame, image_name, (20, 100),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255, 255, 255), 3)
#     cv2.putText(color_frame, f"Target Cell ID {target_cell_id} Visible", (20, 150),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 255), 3)

#     # Run analysis/plotting function
#     analyze_mask_from_path(mask_path=path, frame=frame_idx)

#     # Upscale
#     color_frame = cv2.resize(color_frame,
#                              (int(w * upscale_factor), int(h * upscale_factor)),
#                              interpolation=cv2.INTER_NEAREST)
#     frames.append(color_frame)

# # Save video
# if frames:
#     height, width, _ = frames[0].shape
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
#     for frame in frames:
#         out.write(frame)
#     out.release()
#     print(f"✅ Final video saved: {output_video}")
# else:
#     print("❌ No frames found for target cell.")

# # Save frame list
# with open(output_frame_list_file, "w") as f:
#     for frame_idx, image_name in target_frames:
#         f.write(f"Frame {frame_idx:03d} - {image_name}\n")

# print(f"✅ Accurate frame list saved: {output_frame_list_file}")
# print(f"✅ Total frames with target cell: {len(target_frames)}")

# Modify your tracking code to also store original mask label alongside assigned tracking ID in CSV?
# OR update your video generation code to use the CSV directly?

In [14]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_based_tracking.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)

# Extract data for all cells & target cell
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

# Get list of frames where target cell appears
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []

# Prepare video frames
frames = []

# Map all frame image files
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    frame_num = int(''.join(filter(str.isdigit, os.path.basename(path))))
    mask_map[frame_num] = path

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    # Read image
    mask = cv2.imread(mask_map[frame_num], cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Get all cells in this frame from CSV
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cid == target_cell_id:
            # Highlight target cell
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Draw trajectory of target cell
    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    # Add frame label
    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    # Upscale
    frame_bgr = cv2.resize(frame_bgr, (int(w*upscale_factor), int(h*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video ===
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Video saved: {output_video}")
else:
    print("❌ No frames with target cell found, video not generated.")

# === Save target frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")



✅ Video saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CSV_based_tracking.avi
✅ Target cell frame list saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CSV_frame_list.txt
✅ Total frames with target cell: 69


[mpeg4 @ 0x498b9800] dimensions too large for MPEG-4
[ERROR:0@5282.397] global cap_ffmpeg_impl.hpp:3194 open Could not open codec mpeg4, error: Unspecified error (-22)
[ERROR:0@5282.397] global cap_ffmpeg_impl.hpp:3211 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter

(python:1444257): GStreamer-CRITICAL **: 06:46:29.218: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@5282.397] global cap_gstreamer.cpp:2617 open OpenCV | GStreamer warning: cannot link elements


In [15]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_based_tracking.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 2

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)

# Extract data for all cells & target cell
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

# Get list of frames where target cell appears
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []

# Prepare video frames
frames = []

# Map all frame image files
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    frame_bgr = cv2.resize(frame_bgr, (int(w*upscale_factor), int(h*upscale_factor)), interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# Save video
if frames:
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final video saved: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# Save target frame list
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")



✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

[mpeg4 @ 0x4ac87f00] dimensions too large for MPEG-4
[ERROR:0@5481.193] global cap_ffmpeg_impl.hpp:3194 open Could not open codec mpeg4, error: Unspecified error (-22)
[ERROR:0@5481.193] global cap_ffmpeg_impl.hpp:3211 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter

(python:1444257): GStreamer-CRITICAL **: 06:49:48.014: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@5481.193] global cap_gstreamer.cpp:2617 open OpenCV | GStreamer warning: cannot link elements


In [16]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob


# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_tracking_MJPG.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 1.5  # Reduced to avoid codec error

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()

target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []
frames = []

mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])

        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            cv2.circle(frame_bgr, (cx, cy), 10, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+10, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)
            trajectory_points.append((cx, cy))
        else:
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    frame_bgr = cv2.resize(frame_bgr,
                           (int(w*upscale_factor), int(h*upscale_factor)),
                           interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video with MJPG codec ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Video dimensions: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final video saved using MJPG codec: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# Save target frame list
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

In [3]:
import os
import cv2
import pandas as pd
import numpy as np
from glob import glob

# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_tracking_MJPG_Enhanced.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1
upscale_factor = 1.5

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_all = df.copy()
df_target = df_all[df_all['Cell Number'] == target_cell_id].copy()
target_frames = sorted(df_target['Frame'].unique())
trajectory_points = []
frames = []

# === Map all frame image files ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Failed to parse frame number from: {path}")

# === Process frames ===
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in masks, skipping...")
        continue

    print(f"✅ Drawing Cell {target_cell_id} in Frame {frame_num}")
    mask_path = mask_map[frame_num]
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask for frame {frame_num}")
        continue

    h, w = mask.shape
    frame_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    df_frame = df_all[df_all['Frame'] == frame_num]

    for _, row in df_frame.iterrows():
        cid = int(row['Cell Number'])
        cx, cy = int(row['Centroid_X_MA']), int(row['Centroid_Y_MA'])
        if cx >= w or cy >= h or cx < 0 or cy < 0:
            print(f"⚠️ Skipping invalid centroid {cx},{cy} for Cell {cid} in Frame {frame_num}")
            continue

        if cid == target_cell_id:
            # Simulate visual cell area
            radius = 25  # Or dynamic: int(np.sqrt(row["Area_MA"]) / 2)
            cv2.circle(frame_bgr, (cx, cy), radius, (0, 0, 255), 3)
            cv2.circle(frame_bgr, (cx, cy), 12, (255, 255, 255), -1)
            cv2.putText(frame_bgr, f"{cid}", (cx+12, cy-12),
                        cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 5)
            trajectory_points.append((cx, cy))
        else:
            font_size = 1.0
            cv2.putText(frame_bgr, f"{cid}", (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 255, 0), 2)

    for i in range(1, len(trajectory_points)):
        cv2.line(frame_bgr, trajectory_points[i-1], trajectory_points[i], (0, 0, 255), 2)

    cv2.putText(frame_bgr, f"Frame {frame_num:03d}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255, 255, 255), 4)

    frame_bgr = cv2.resize(frame_bgr,
                           (int(w*upscale_factor), int(h*upscale_factor)),
                           interpolation=cv2.INTER_NEAREST)
    frames.append(frame_bgr)

# === Save video with MJPG codec ===
if frames:
    height, width, _ = frames[0].shape
    print(f"Video dimensions: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print(f"✅ Final enhanced video saved: {output_video}")
else:
    print("❌ No frames successfully drawn. Video not generated.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Target cell frame list saved: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


✅ Drawing Cell 4 in Frame 2
✅ Drawing Cell 4 in Frame 3
✅ Drawing Cell 4 in Frame 4
✅ Drawing Cell 4 in Frame 6
✅ Drawing Cell 4 in Frame 7
✅ Drawing Cell 4 in Frame 8
✅ Drawing Cell 4 in Frame 10
✅ Drawing Cell 4 in Frame 11
✅ Drawing Cell 4 in Frame 19
✅ Drawing Cell 4 in Frame 20
✅ Drawing Cell 4 in Frame 21
✅ Drawing Cell 4 in Frame 23
✅ Drawing Cell 4 in Frame 24
✅ Drawing Cell 4 in Frame 25
✅ Drawing Cell 4 in Frame 26
✅ Drawing Cell 4 in Frame 28
✅ Drawing Cell 4 in Frame 29
✅ Drawing Cell 4 in Frame 30
✅ Drawing Cell 4 in Frame 31
✅ Drawing Cell 4 in Frame 32
✅ Drawing Cell 4 in Frame 33
✅ Drawing Cell 4 in Frame 34
✅ Drawing Cell 4 in Frame 35
✅ Drawing Cell 4 in Frame 36
✅ Drawing Cell 4 in Frame 37
✅ Drawing Cell 4 in Frame 38
✅ Drawing Cell 4 in Frame 43
✅ Drawing Cell 4 in Frame 45
✅ Drawing Cell 4 in Frame 46
✅ Drawing Cell 4 in Frame 47
✅ Drawing Cell 4 in Frame 48
✅ Drawing Cell 4 in Frame 50
✅ Drawing Cell 4 in Frame 51
✅ Drawing Cell 4 in Frame 52
✅ Drawing Cell 4 in 

In [18]:
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         # plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

In [4]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import imageio
import random

# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
temp_plot_dir = os.path.join(output_dir, "temp_plots")
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_Matplotlib_Colored_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_target = df[df["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Create mapping of frame to mask file (use index method) ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for idx, path in enumerate(mask_paths):
    frame_num = idx + 1  # Assume index matches frame number
    mask_map[frame_num] = path

# === Assign distinct colors to other cells ===
cell_ids = df["Cell Number"].unique()
cell_colors = {}
for cid in cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = "red"
    else:
        color = "#" + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
        cell_colors[cid] = color

# === Define plot_labels function with distinct colors ===
def plot_labels(original, ax, frame_num, txt_args_base, title=None, cmap='gray'):
    ax.imshow(original, cmap=cmap)
    labels = np.unique(original)
    print(f"Labels in plot_labels function for frame {frame_num}: {labels}")
    ax.set_title(title)
    for label in labels:
        if label != 0:
            inds = np.argwhere(original == label)
            loc = inds.mean(0)
            txt_args = txt_args_base.copy()
            txt_args['color'] = cell_colors.get(label, 'white')
            txt_args['fontsize'] = 'xx-large' if label == target_cell_id else 'large'
            ax.text(loc[1], loc[0], str(label), **txt_args)
    ax.axis("off")

# === Create labeled frames using matplotlib ===
plot_frame_paths = []
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask = imageio.imread(mask_map[frame_num])
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_labels(
        mask, ax, frame_num,
        txt_args_base={'ha': 'center', 'va': 'center', 'fontweight': 'bold'},
        title=f"Frame {frame_num} - Cell {target_cell_id}"
    )
    plot_frame_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_frame_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_frame_path)

# === Save final video ===
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for frame_img_path in plot_frame_paths:
        img = cv2.imread(frame_img_path)
        out.write(img)
    out.release()
    print(f"✅ Final enhanced colorful video saved: {output_video}")
else:
    print("❌ No video frames generated. Video not saved.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Frame list saved to: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


/tmp/ipykernel_1517116/4120977095.py:69: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask = imageio.imread(mask_map[frame_num])


Labels in plot_labels function for frame 2: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 3: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 6: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 7: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 8: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame 10: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame 11: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame 19: [ 0  2  3  4  6  8  9 10 11 39 40]
Labels in plot_labels function for frame 20: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]
Labels in plot_labels function for frame 21: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]
Labels in plot_labels function for frame 23: [ 0  2  3  4  6  8  9 10 11

In [19]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import imageio

# === PARAMETERS ===
# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

temp_plot_dir = os.path.join(output_dir, "temp_plots")
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_Matplotlib_Labeled_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_target = df[df["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Create mapping of frame to mask file ===
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {}
for path in mask_paths:
    try:
        fname = os.path.basename(path)
        digits = ''.join(filter(str.isdigit, fname))
        frame_num = int(digits)
        mask_map[frame_num] = path
    except:
        print(f"⚠️ Could not parse frame number from {path}")

# === Define plot_labels function ===
def plot_labels(original, ax, title=None,
                txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
                          'fontsize': 'xx-large', 'fontweight': 'bold'},
                cmap='gist_earth'):
    ax.imshow(original, cmap=cmap)
    labels = np.unique(original)
    print(f"Labels in plot_labels function for frame {title}: {labels}")
    ax.set_title(title)
    for label in labels:
        if label != 0:
            inds = np.argwhere(original == label)
            loc = inds.mean(0)
            ax.text(loc[1], loc[0], str(label), **txt_args)
    ax.axis("off")

# === Create frames using matplotlib ===
plot_frame_paths = []
for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask = imageio.imread(mask_map[frame_num])
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_labels(mask, ax, title=f"Frame {frame_num} - Cell {target_cell_id}")
    plot_frame_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_frame_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_frame_path)

# === Read images and write video ===
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for frame_img_path in plot_frame_paths:
        img = cv2.imread(frame_img_path)
        out.write(img)
    out.release()
    print(f"✅ Final enhanced matplotlib-labeled video saved: {output_video}")
else:
    print("❌ No video frames generated. Video not saved.")

# === Save frame list ===
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Frame list saved to: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")


/tmp/ipykernel_1444257/2430220196.py:65: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask = imageio.imread(mask_map[frame_num])


Labels in plot_labels function for frame Frame 2 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 3 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 4 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14 22]
Labels in plot_labels function for frame Frame 6 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 7 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 14]
Labels in plot_labels function for frame Frame 8 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame Frame 10 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12]
Labels in plot_labels function for frame Frame 11 - Cell 4: [ 0  2  3  4  5  6  7  8  9 10 11 12 27]
Labels in plot_labels function for frame Frame 19 - Cell 4: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]
Labels in plot_labels function for frame Frame 20 - Cell 4: [ 0  2  3  4  6  8  9 10 11 39 40 43 44]

In [ ]:
# def analyze_mask_from_path(mask_path, frame):
#     def plot_labels(original, ax, title=None,
#                     txt_args={'color': 'red', 'ha': 'center', 'va': 'center',
#                               'fontsize': 'x-large', 'fontweight': 'bold'},
#                     cmap='gist_earth'):
#         ax.imshow(original, cmap=cmap)
#         labels = np.unique(original)
#         print("Labels in plot_labels function:", labels)
#         ax.set_title(title)
#         for label in labels:
#             if label != 0:
#                 inds = np.argwhere(original == label)
#                 loc = inds.mean(0)
#                 ax.text(loc[1], loc[0], str(label), **txt_args)

#     try:
#         mask = imageio.imread(mask_path)
#         unique_labels = np.unique(mask)
#         print(f"Frame {frame} - Labels from the mask: {unique_labels}")
#         fig, ax = plt.subplots(figsize=(10, 10))
#         plot_labels(mask, ax, title=f"Labeled Mask (Frame {frame})")
#         # plt.show()
#     except Exception as e:
#         print(f"❌ Failed to analyze mask for frame {frame}: {e}")

# csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"


# === PARAMETERS ===
# csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
# image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
# output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

import os
import cv2
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import imageio
from skimage.measure import regionprops, label
from scipy.spatial import distance

# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"
temp_plot_dir = os.path.join(output_dir, "temp_plots")

target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_Matched_Mask_Label_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_Matched_Frame_List.txt")
video_fps = 1
distance_threshold = 50  # max pixel distance to associate CSV cell to mask region

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_target = df[df["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Map mask files using sorted order (index-based map)
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
# mask_map = {idx + 1: path for idx, path in enumerate(mask_paths)}


mask_map = {}
for path in mask_paths:
    fname = os.path.basename(path)
    frame_match = re.search(r'(\d{2,4})', fname)
    if frame_match:
        frame_num = int(frame_match.group(1))
        mask_map[frame_num] = path

# === Define mask label plotting function with matched highlight
def plot_matched_labels(mask, ax, matched_label=None,
                        txt_args={'color': 'cyan', 'ha': 'center', 'va': 'center',
                                  'fontsize': 'large', 'fontweight': 'bold'},
                        highlight_args={'color': 'red', 'ha': 'center', 'va': 'center',
                                        'fontsize': 'xx-large', 'fontweight': 'bold'},
                        cmap='gist_earth'):
    ax.imshow(mask, cmap=cmap)
    labels = np.unique(mask)
    ax.set_title(f"Cell {target_cell_id} Matched in Mask")
    for label_id in labels:
        if label_id == 0:
            continue
        coords = np.argwhere(mask == label_id)
        centroid = coords.mean(axis=0)
        label_txt_args = highlight_args if label_id == matched_label else txt_args
        ax.text(centroid[1], centroid[0], str(label_id), **label_txt_args)
    ax.axis("off")

# === Create frames by matching CSV Cell4 with mask region
plot_frame_paths = []
matched_frame_ids = []

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask_img = imageio.imread(mask_map[frame_num])
    labeled_mask = label(mask_img)
    regions = regionprops(labeled_mask)

    # Get CSV centroid for Cell 4 in this frame
    row = df_target[df_target["Frame"] == frame_num]
    if row.empty:
        continue
    cx_csv = row["Centroid_X_MA"].values[0]
    cy_csv = row["Centroid_Y_MA"].values[0]

    # Find closest mask label to CSV Cell 4 centroid
    min_dist = float("inf")
    matched_label = None
    for region in regions:
        y, x = region.centroid
        dist = distance.euclidean((cx_csv, cy_csv), (x * 0.68, y * 0.68))  # account for scaling
        if dist < min_dist and dist <= distance_threshold:
            min_dist = dist
            matched_label = region.label

    if matched_label is None:
        print(f"⚠️ Cell 4 NOT matched in frame {frame_num} (no region close enough)")
        continue

    matched_frame_ids.append(frame_num)

    # Plot and save frame
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_matched_labels(labeled_mask, ax, matched_label=matched_label)
    plot_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_path)

# === Assemble final video
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for img_path in plot_frame_paths:
        img = cv2.imread(img_path)
        out.write(img)
    out.release()
    print(f"✅ Final Cell4 matched mask-label video saved: {output_video}")
else:
    print("❌ No frames drawn. Video not created.")

# === Save matched frame list
with open(output_frame_list, "w") as f:
    for frame_num in matched_frame_ids:
        f.write(f"{frame_num}\n")

print(f"✅ Matched frame list saved to: {output_frame_list}")
print(f"✅ Total frames with matched Cell 4: {len(matched_frame_ids)}")






/tmp/ipykernel_1517116/4264189400.py:99: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask_img = imageio.imread(mask_map[frame_num])


⚠️ Cell 4 NOT matched in frame 6 (no region close enough)
⚠️ Cell 4 NOT matched in frame 23 (no region close enough)
⚠️ Cell 4 NOT matched in frame 58 (no region close enough)
⚠️ Cell 4 NOT matched in frame 73 (no region close enough)
⚠️ Cell 4 NOT matched in frame 76 (no region close enough)
⚠️ Cell 4 NOT matched in frame 80 (no region close enough)
⚠️ Cell 4 NOT matched in frame 86 (no region close enough)
Video frame size: 292x656
✅ Final Cell4 matched mask-label video saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_Matched_Mask_Label_Video.avi
✅ Matched frame list saved to: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_Matched_Frame_List.txt
✅ Total frames with matched Cell 4: 62


In [7]:
# Re-execute the final code after environment reset

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

# === PARAMETERS ===
# === PARAMETERS ===
csv_path = "/home/MinaHossain/EmbedTrack/PCA_t-SNE_PHATE/Cells_Centroid_Velocity_MA_5.csv"  # <<< Adjust if needed
image_dir = "/home/MinaHossain/EmbedTrack/Shape_measure/HP4_TRA"  # Directory containing raw mask images
output_dir = "/home/MinaHossain/EmbedTrack/Cell_Frame-Video"

temp_plot_dir = os.path.join(output_dir, "temp_plots")
target_cell_id = 4
output_video = os.path.join(output_dir, f"Cell_{target_cell_id}_CentroidOverlay_Video.avi")
output_frame_list = os.path.join(output_dir, f"Cell_{target_cell_id}_CSV_frame_list.txt")
video_fps = 1

# === Ensure directories exist ===
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_plot_dir, exist_ok=True)

# === Load CSV data ===
df = pd.read_csv(csv_path)
df_all = df.copy()
df_target = df_all[df_all["Cell Number"] == target_cell_id]
target_frames = sorted(df_target["Frame"].unique())

# === Map mask images by sorted order (assume index-based matching)
mask_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
mask_map = {idx + 1: path for idx, path in enumerate(mask_paths)}

# === Assign distinct colors to all cells
cell_ids = df_all["Cell Number"].unique()
cell_colors = {}
for cid in cell_ids:
    if cid == target_cell_id:
        cell_colors[cid] = "red"
    else:
        color = "#" + ''.join(np.random.choice(list('0123456789ABCDEF'), 6))
        cell_colors[cid] = color

# === Create frames using matplotlib overlaying centroid labels
plot_frame_paths = []

for frame_num in target_frames:
    if frame_num not in mask_map:
        print(f"❌ Frame {frame_num} not found in mask map.")
        continue

    mask_img = cv2.imread(mask_map[frame_num], cv2.IMREAD_GRAYSCALE)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(mask_img, cmap='gray')
    ax.set_title(f"Frame {frame_num}")
    ax.axis("off")

    df_frame = df_all[df_all["Frame"] == frame_num]
    for _, row in df_frame.iterrows():
        cid = int(row["Cell Number"])
        cx, cy = row["Centroid_X_MA"], row["Centroid_Y_MA"]
        color = cell_colors[cid]
        size = 24 if cid == target_cell_id else 14
        weight = "bold" if cid == target_cell_id else "normal"
        ax.text(cx, cy, str(cid), color=color,
                fontsize=size, fontweight=weight,
                ha='center', va='center')

    plot_path = os.path.join(temp_plot_dir, f"frame_{frame_num:03d}.png")
    fig.savefig(plot_path, bbox_inches="tight")
    plt.close(fig)
    plot_frame_paths.append(plot_path)

# === Assemble video
if plot_frame_paths:
    sample_img = cv2.imread(plot_frame_paths[0])
    height, width, _ = sample_img.shape
    print(f"Video frame size: {width}x{height}")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video, fourcc, video_fps, (width, height))

    for img_path in plot_frame_paths:
        img = cv2.imread(img_path)
        out.write(img)
    out.release()
    print(f"✅ Final CSV-based centroid-labeled video saved: {output_video}")
else:
    print("❌ No frames drawn. Video not created.")

# === Save frame list
with open(output_frame_list, "w") as f:
    for frame_num in target_frames:
        f.write(f"{frame_num}\n")

print(f"✅ Frame list saved to: {output_frame_list}")
print(f"✅ Total frames with target cell: {len(target_frames)}")



Video frame size: 292x656
✅ Final CSV-based centroid-labeled video saved: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CentroidOverlay_Video.avi
✅ Frame list saved to: /home/MinaHossain/EmbedTrack/Cell_Frame-Video/Cell_4_CSV_frame_list.txt
✅ Total frames with target cell: 69


# New Analysis

In [ ]:
# Re-import dependencies after code execution environment reset
# can Modify your tracking code to also store original mask label alongside assigned tracking ID in CSV?
import os
import cv2
import numpy as np
import pandas as pd
from skimage.measure import label, regionprops
import time
import re
import matplotlib.pyplot as plt

# === PARAMETERS ===
mask_directory = "./HP4_TRA"  # Change this to your mask path
output_directory = "./Tracking_Output"
size_threshold = 1000
max_centroid_dist = 20
window_size = 5

# === Ensure output directory exists ===
os.makedirs(output_directory, exist_ok=True)

start_time = time.time()

def calculate_shape_factors(region):
    area = region.area
    perimeter = region.perimeter
    convex_area = region.convex_area
    bounding_box_area = (region.bbox[2] - region.bbox[0]) * (region.bbox[3] - region.bbox[1])
    solidity = area / convex_area if convex_area > 0 else 0
    extent = area / bounding_box_area if bounding_box_area > 0 else 0
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

    convex_mask = region.convex_image.astype(np.uint8) * 255
    contours, _ = cv2.findContours(convex_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    convex_perimeter = sum(cv2.arcLength(cnt, True) for cnt in contours) if contours else 0
    convexity = convex_perimeter / perimeter if perimeter > 0 else 0

    min_diameter = region.minor_axis_length
    max_diameter = region.major_axis_length
    elongation = 1 - (min_diameter / max_diameter) if max_diameter > 0 else 0
    compactness = (np.sqrt(4 * area / np.pi)) / max_diameter if max_diameter > 0 else 0

    centroid = region.centroid
    cx, cy = centroid[1] * 0.68, centroid[0] * 0.68  # Scaled

    return [area, perimeter, extent, solidity, compactness, elongation,
            circularity, convexity, cx, cy]

# === Load masks ===
mask_files = sorted([os.path.join(mask_directory, f) for f in os.listdir(mask_directory) if f.endswith(".tif")])
if not mask_files:
    raise FileNotFoundError(f"No TIFF files found in {mask_directory}")

all_results = []
cell_tracks = {}
previous_centroids = {}
previous_frames = {}

current_cell_id = 1

for mask_path in mask_files:
    frame_str = os.path.splitext(os.path.basename(mask_path))[0]
    frame_match = re.search(r'\d+', frame_str)
    frame = int(frame_match.group()) if frame_match else None
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    if mask is None:
        print(f"Skipping {frame}: Could not read {mask_path}.")
        continue

    labeled_mask = label(mask)
    properties = regionprops(labeled_mask)
    filtered_regions = [prop for prop in properties if prop.label != 0 and prop.area >= size_threshold]

    if not filtered_regions:
        print(f"Skipping frame {frame}: No valid cells detected.")
        continue

    used_cell_ids = set()
    for region in filtered_regions:
        shape_factors = calculate_shape_factors(region)
        centroid = (shape_factors[-2], shape_factors[-1])
        matched_cell_id = None

        # Get original label from the mask
        original_label = np.unique(mask[region.coords[:, 0], region.coords[:, 1]])[0]

        for cell_id, prev_centroid in cell_tracks.items():
            if cell_id not in used_cell_ids:
                x_dist = centroid[0] - prev_centroid[0]
                y_dist = centroid[1] - prev_centroid[1]
                if abs(x_dist) <= max_centroid_dist and abs(y_dist) <= max_centroid_dist:
                    matched_cell_id = cell_id
                    used_cell_ids.add(cell_id)
                    break

        if matched_cell_id is None:
            matched_cell_id = current_cell_id
            used_cell_ids.add(matched_cell_id)
            current_cell_id += 1

        prev_x, prev_y = previous_centroids.get(matched_cell_id, (None, None))
        prev_frame = previous_frames.get(matched_cell_id, None)
        if prev_x is not None and prev_y is not None and prev_frame is not None:
            frame_diff = frame - prev_frame
            x_centroid_distance = centroid[0] - prev_x
            y_centroid_distance = centroid[1] - prev_y
            x_centroid_velocity = x_centroid_distance / frame_diff if frame_diff > 0 else 0
            y_centroid_velocity = y_centroid_distance / frame_diff if frame_diff > 0 else 0
        else:
            x_centroid_distance = y_centroid_distance = 0
            x_centroid_velocity = y_centroid_velocity = 0

        previous_centroids[matched_cell_id] = centroid
        previous_frames[matched_cell_id] = frame
        cell_tracks[matched_cell_id] = centroid

        all_results.append([
            matched_cell_id,
            original_label,
            frame,
            *shape_factors,
            x_centroid_distance,
            y_centroid_distance,
            x_centroid_velocity,
            y_centroid_velocity
        ])

columns = ["Cell Number", "Original Mask Label", "Frame", "Area", "Perimeter", "Extent", "Solidity", "Compactness",
           "Elongation", "Circularity", "Convexity", "Centroid_X", "Centroid_Y",
           "X_Centroid_Distance", "Y_Centroid_Distance", "X_Centroid_Velocity", "Y_Centroid_Velocity"]

df = pd.DataFrame(all_results, columns=columns)

# Save CSV with Original Label Included
csv_path = os.path.join(output_directory, f"Cells_Centroid_With_OriginalLabel.csv")
df.to_csv(csv_path, index=False)

print(f"✅ CSV with original mask labels saved to: {csv_path}")
print(f"✅ Total Cells Tracked: {df['Cell Number'].nunique()}")
print(f"✅ Total Frames Processed: {df['Frame'].nunique()}")
print(f"✅ Execution Time: {time.time() - start_time:.2f} seconds")
